In [118]:
!pip install mysql.connector
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import errorcode

In [119]:
#Loading datasets from GitHub raw urls

code_url = "https://raw.githubusercontent.com/bradley121901/canadian_labour/main/2023-CSV/LFS_PUMF_EPA_FGMD_codebook.csv"
jan_url = "https://raw.githubusercontent.com/bradley121901/canadian_labour/main/2023-CSV/pub0123.csv"
feb_url = "https://raw.githubusercontent.com/bradley121901/canadian_labour/main/2023-CSV/pub0223.csv"
mar_url = "https://raw.githubusercontent.com/bradley121901/canadian_labour/main/2023-CSV/pub0323.csv"

In [120]:
# Loading the codebook numeral to english value translations

codedf = pd.read_csv(code_url)

In [121]:
# Loading datasets

jandf = pd.read_csv(jan_url)
febdf = pd.read_csv(feb_url)
mardf = pd.read_csv(mar_url)

In [ ]:
# Merge datasets
surveydf = pd.concat([jandf, febdf, mardf], ignore_index=True)
surveydf

,REC_NUM,SURVYEAR,SURVMNTH,LFSSTAT,PROV,CMA,AGE_12,AGE_6,SEX,MARSTAT,...,LKATADS,LKANSADS,LKOTHERN,PRIORACT,YNOLOOK,TLOLOOK,SCHOOLN,EFAMTYPE,AGYOWNK,FINALWT
0,1,2023,1,1,24,2,7,NaN,2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,1312
1,2,2023,1,1,47,0,4,NaN,2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,91
2,3,2023,1,1,10,0,10,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5,NaN,136
3,4,2023,1,4,35,0,12,NaN,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN,107
4,5,2023,1,1,35,0,5,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,18,NaN,242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313953,101039,2023,3,3,59,0,5,NaN,1,6,...,NaN,1.0,1.0,0.0,NaN,NaN,1.0,1,NaN,849
313954,101040,2023,3,1,46,6,8,NaN,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3,3.0,143
313955,101041,2023,3,1,24,0,10,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5,NaN,135
313956,101042,2023,3,1,35,4,3,5.0,2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2,NaN,1192


In [ ]:
# Create dictionary where key is the column to be translated, and value is a tuple storing the numeral to english mapping
# The first item of the tuple is the numerical value
# The second item of the tuple is the english translation

d = {}
temp = None
for index, row in codedf.iterrows():
    if pd.notnull(row['Position_Position']):
        temp = row['Variable_Variable']
        d[temp] = {}
    else:
        if '-' not in str(row['Variable_Variable']):
            if row['Variable_Variable'] != "0" and row['Variable_Variable'] != 0:
                d[temp][row['Variable_Variable'].lstrip('0')] = row['EnglishLabel_EtiquetteAnglais']
            else:
                d[temp][row['Variable_Variable']] = row['EnglishLabel_EtiquetteAnglais']
        else:
            d[temp]["blank"] = "blank"

mapping_dict = {key: value for key, value in d.items() if value != {'blank': 'blank'} and value !={'blank': 'Not applicable'}}
print(mapping_dict)

{'survmnth': {'1': 'January', '2': 'February', '3': 'March', '4': 'April', '5': 'May', '6': 'June', '7': 'July', '8': 'August', '9': 'September', '10': 'October', '11': 'November', '12': 'December'}, ' lfsstat    ': {'1': 'Employed, at work', '2': 'Employed, absent from work', '3': 'Unemployed', '4': 'Not in labour force'}, 'prov': {'10': 'Newfoundland and Labrador', '11': 'Prince Edward Island', '12': 'Nova Scotia', '13': 'New Brunswick', '24': 'Quebec', '35': 'Ontario', '46': 'Manitoba', '47': 'Saskatchewan', '48': 'Alberta', '59': 'British Columbia'}, 'cma': {'1': 'QuÈbec', '2': 'MontrÈal', '3': 'OttawañGatineau (Ontario part)', '4': 'Toronto', '5': 'Hamilton', '6': 'Winnipeg', '7': 'Calgary', '8': 'Edmonton', '9': 'Vancouver', '0': 'Other CMA or non-CMA'}, 'age_12': {'1': '15 to 19 years', '2': '20 to 24 years', '3': '25 to 29 years', '4': '30 to 34 years', '5': '35 to 39 years', '6': '40 to 44 years', '7': '45 to 49 years', '8': '50 to 54 years', '9': '55 to 59 years', '10': '60 t

In [ ]:
# Apply english mapping conversions to dataset

for c_name in surveydf.columns:
    if c_name.lower() in mapping_dict:
        if surveydf[c_name].dtype == 'float64':
            surveydf[c_name] = pd.to_numeric(surveydf[c_name], errors='coerce').astype('Int64')  # Convert to nullable integer
        surveydf[c_name] = surveydf[c_name].astype(str)
        surveydf[c_name] = surveydf[c_name].map(mapping_dict[c_name.lower()])

surveydf

,REC_NUM,SURVYEAR,SURVMNTH,LFSSTAT,PROV,CMA,AGE_12,AGE_6,SEX,MARSTAT,...,LKATADS,LKANSADS,LKOTHERN,PRIORACT,YNOLOOK,TLOLOOK,SCHOOLN,EFAMTYPE,AGYOWNK,FINALWT
0,1,2023,January,1,Quebec,MontrÈal,45 to 49 years,NaN,Female,"Single, never married",...,NaN,NaN,NaN,NaN,NaN,NaN,Non-student,Person not in an economic family,NaN,1312
1,2,2023,January,1,Saskatchewan,Other CMA or non-CMA,30 to 34 years,NaN,Female,"Single, never married",...,NaN,NaN,NaN,NaN,NaN,NaN,Non-student,Person not in an economic family,NaN,91
2,3,2023,January,1,Newfoundland and Labrador,Other CMA or non-CMA,60 to 64 years,NaN,Male,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,Non-student,"Single-earner couple, male employed, no childr...",NaN,136
3,4,2023,January,4,Ontario,Other CMA or non-CMA,70 and over,NaN,Female,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non-earner couple, no children or none under 25",NaN,107
4,5,2023,January,1,Ontario,Other CMA or non-CMA,35 to 39 years,NaN,Male,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,Non-student,Other families,NaN,242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313953,101039,2023,March,3,British Columbia,Other CMA or non-CMA,35 to 39 years,NaN,Male,"Single, never married",...,NaN,Yes,1.0,Other,NaN,NaN,Non-student,Person not in an economic family,NaN,849
313954,101040,2023,March,1,Manitoba,Winnipeg,50 to 54 years,NaN,Female,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,Non-student,"Dual-earner couple, youngest child 0 to 17",Youngest child 13 to 17 years,143
313955,101041,2023,March,1,Quebec,Other CMA or non-CMA,60 to 64 years,NaN,Male,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,Non-student,"Single-earner couple, male employed, no childr...",NaN,135
313956,101042,2023,March,1,Ontario,Toronto,25 to 29 years,5.0,Female,"Single, never married",...,NaN,NaN,NaN,NaN,NaN,NaN,Non-student,"Dual-earner couple, no children or none under 25",NaN,1192


In [ ]:
# Converting to dollar digits
surveydf['HRLYEARN'] = surveydf['HRLYEARN'] * 0.01
surveydf['HRLYEARN']

0         20.00
1         42.38
2         19.00
3           NaN
4         34.83
          ...  
313953      NaN
313954    23.28
313955    64.10
313956    32.97
313957      NaN
Name: HRLYEARN, Length: 313958, dtype: float64

In [ ]:
# Convert hours worked per week to correct decimal place
surveydf['UTOTHRS'] = surveydf['UTOTHRS']*0.1
surveydf['UTOTHRS']

0         40.0
1         42.5
2         40.0
3          NaN
4         40.0
          ... 
313953     NaN
313954    40.0
313955    37.5
313956    35.0
313957     NaN
Name: UTOTHRS, Length: 313958, dtype: float64

In [ ]:
# creating new feature weekly income from hourly salary with usual hours worked
surveydf['WEEKINCOME'] = surveydf['HRLYEARN'] * surveydf['UTOTHRS']
surveydf['WEEKINCOME']

0          800.00
1         1801.15
2          760.00
3             NaN
4         1393.20
           ...   
313953        NaN
313954     931.20
313955    2403.75
313956    1153.95
313957        NaN
Name: WEEKINCOME, Length: 313958, dtype: float64

In [ ]:
surveydf['ANNUALINCOME'] = surveydf['WEEKINCOME'] * 52
surveydf['ANNUALINCOME']

0          41600.0
1          93659.8
2          39520.0
3              NaN
4          72446.4
            ...   
313953         NaN
313954     48422.4
313955    124995.0
313956     60005.4
313957         NaN
Name: ANNUALINCOME, Length: 313958, dtype: float64

In [ ]:
# dictionary to map weekly income to provincial tax bracket
tax_brackets_province = {
            "Newfoundland and Labrador": {(0, 702): 1, (703, 1404): 2, (1405, 2229): 3, (2230, float('inf')): 4},
            "Prince Edward Island": {(0, 732): 1, (733, 1465): 2, (1466, 2140): 3, (2141, float('inf')): 4},
            "Nova Scotia": {(0, 749): 1, (750, 1498): 2, (1499, 2915): 3, (2916, float('inf')): 4},
            "New Brunswick": {(0, 738): 1, (739, 1476): 2, (1477, 2293): 3, (2294, float('inf')): 4},
            "Quebec": {(0, 704): 1, (705, 1408): 2, (1409, 2181): 3, (2182, float('inf')): 4},
            "Ontario": {(0, 787): 1, (788, 1574): 2, (1575, 2885): 3, (2886, float('inf')): 4},
            "Manitoba": {(0, 740): 1, (741, 1482): 2, (1483, 2144): 3, (2145, float('inf')): 4},
            "Saskatchewan": {(0, 745): 1, (746, 1490): 2, (1491, 2173): 3, (2174, float('inf')): 4},
            "Alberta": {(0, 734): 1, (735, 1470): 2, (1471, 2107): 3, (2108, float('inf')): 4},
            "British Columbia": {(0, 775): 1, (776, 1550): 2, (1551, 2250): 3, (2251, float('inf')): 4}}

def get_tax_bracket(province, salary):
    province_tax_brackets = tax_brackets_province.get(province)
    if province_tax_brackets:
        for x, y in province_tax_brackets.items():
            if x[0] <= salary <= x[1]:
                return y
    else:
        return None

surveydf['TAXBRAC'] = surveydf.apply(lambda x: get_tax_bracket(x['PROV'], x['WEEKINCOME']), axis=1)
surveydf['TAXBRAC']

0         2.0
1         3.0
2         2.0
3         NaN
4         2.0
         ... 
313953    NaN
313954    2.0
313955    4.0
313956    2.0
313957    NaN
Name: TAXBRAC, Length: 313958, dtype: float64

In [ ]:
survey_desc_fill = ["YABSENT", "PROV", "CMA", "AGE_6","AGE_12","SEX", 'MARSTAT', 'EDUC', 'SCHOOLN', 'IMMIG', 'EFAMTYPE', 'AGYOWNK', 'TAXBRAC', 'SURVYEAR','SURVMNTH', 'NAICS_21', 'NOC_10', 'NOC_43', 'ESTSIZE', 'FIRMSIZE', 'WEEKINCOME', 'PERMTEMP']
for column in survey_desc_fill:
    # Calculate the most frequent value
    mode_value = surveydf[column].mode()[0]
    # Replace NaN with the most frequent value
    surveydf[column].fillna(mode_value, inplace=True)

In [ ]:
survey_num_fill = ['LFSSTAT', 'HRLYEARN', 'UHRSMAIN', 'AHRSMAIN', 'UTOTHRS', 'ATOTHRS', 'HRSAWAY', 'WKSAWAY', 'PAIDOT', 'XTRAHRS', 'DURUNEMP', 'TENURE', 'PREVTEN', 'ANNUALINCOME']
for column in survey_num_fill:
                # Calculate average of numerical values in the column
                avg_value = surveydf[column].mean()
                # Replace NaN with the average value
                surveydf[column].fillna(avg_value, inplace=True)

In [ ]:
surveydf

,REC_NUM,SURVYEAR,SURVMNTH,LFSSTAT,PROV,CMA,AGE_12,AGE_6,SEX,MARSTAT,...,PRIORACT,YNOLOOK,TLOLOOK,SCHOOLN,EFAMTYPE,AGYOWNK,FINALWT,WEEKINCOME,ANNUALINCOME,TAXBRAC
0,1,2023,January,1,Quebec,MontrÈal,45 to 49 years,6.0,Female,"Single, never married",...,NaN,NaN,NaN,Non-student,Person not in an economic family,Youngest child less than 6 years,1312,800.00,41600.000000,2.0
1,2,2023,January,1,Saskatchewan,Other CMA or non-CMA,30 to 34 years,6.0,Female,"Single, never married",...,NaN,NaN,NaN,Non-student,Person not in an economic family,Youngest child less than 6 years,91,1801.15,93659.800000,3.0
2,3,2023,January,1,Newfoundland and Labrador,Other CMA or non-CMA,60 to 64 years,6.0,Male,Married,...,NaN,NaN,NaN,Non-student,"Single-earner couple, male employed, no childr...",Youngest child less than 6 years,136,760.00,39520.000000,2.0
3,4,2023,January,4,Ontario,Other CMA or non-CMA,70 and over,6.0,Female,Married,...,NaN,NaN,NaN,Non-student,"Non-earner couple, no children or none under 25",Youngest child less than 6 years,107,1000.00,62935.340129,2.0
4,5,2023,January,1,Ontario,Other CMA or non-CMA,35 to 39 years,6.0,Male,Married,...,NaN,NaN,NaN,Non-student,Other families,Youngest child less than 6 years,242,1393.20,72446.400000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313953,101039,2023,March,3,British Columbia,Other CMA or non-CMA,35 to 39 years,6.0,Male,"Single, never married",...,Other,NaN,NaN,Non-student,Person not in an economic family,Youngest child less than 6 years,849,1000.00,62935.340129,2.0
313954,101040,2023,March,1,Manitoba,Winnipeg,50 to 54 years,6.0,Female,Married,...,NaN,NaN,NaN,Non-student,"Dual-earner couple, youngest child 0 to 17",Youngest child 13 to 17 years,143,931.20,48422.400000,2.0
313955,101041,2023,March,1,Quebec,Other CMA or non-CMA,60 to 64 years,6.0,Male,Married,...,NaN,NaN,NaN,Non-student,"Single-earner couple, male employed, no childr...",Youngest child less than 6 years,135,2403.75,124995.000000,4.0
313956,101042,2023,March,1,Ontario,Toronto,25 to 29 years,5.0,Female,"Single, never married",...,NaN,NaN,NaN,Non-student,"Dual-earner couple, no children or none under 25",Youngest child less than 6 years,1192,1153.95,60005.400000,2.0


In [ ]:
labour_force_fact = surveydf[['REC_NUM', 'LFSSTAT', 'HRLYEARN', 'UHRSMAIN', 'AHRSMAIN', 'UTOTHRS', 'ATOTHRS', 'HRSAWAY', 'WKSAWAY', 'PAIDOT', 'XTRAHRS', 'YABSENT', 'DURUNEMP', 'TENURE', 'PREVTEN', 'WEEKINCOME', 'ANNUALINCOME']]
date = surveydf[[ 'SURVYEAR','SURVMNTH']]
geographic = surveydf[[ 'PROV', 'CMA']]
demographic = surveydf[[ 'AGE_12', 'AGE_6', 'SEX', 'MARSTAT', 'EDUC', 'SCHOOLN', 'IMMIG', 'EFAMTYPE', 'AGYOWNK', 'TAXBRAC']]
job = surveydf[['NAICS_21', 'NOC_10', 'NOC_43', 'MJH', 'PERMTEMP', 'ESTSIZE', 'FIRMSIZE', 'UNION','WHYPT']]
unemployed = surveydf[['PRIORACT','FLOWUNEM', 'LKPUBAG', 'LKATADS', 'WHYLEFTO', 'WHYLEFTN','YNOLOOK', 'UNEMFTPT']]


In [ ]:
def suggorate_key_pipeline(df, key_name):
            df = df.drop_duplicates().copy()
            df[key_name]= range(1, len(df) + 1)
            columns_except_key_name = [col for col in df.columns if col != key_name]
            labour_with_key = pd.merge(surveydf, df, on=columns_except_key_name, how='left')
            labour_with_key.drop(columns_except_key_name, axis=1, inplace=True)
            labour_force_fact.loc[:, key_name] = labour_with_key[key_name].values
            return df

In [ ]:
date = suggorate_key_pipeline(date, "DATE_KEY")
geographic = suggorate_key_pipeline(geographic, "GEOGRAPHIC_KEY")
demographic = suggorate_key_pipeline(demographic, "DEMOGRAPHIC_KEY")
job = suggorate_key_pipeline(job, "JOB_KEY")
unemployed = suggorate_key_pipeline(unemployed, "UNEMPLOYED_KEY")

/var/folders/jb/pfrlwvtn54x18ynnfy_9tkg00000gn/T/ipykernel_26727/2235184035.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labour_force_fact.loc[:, key_name] = labour_with_key[key_name].values
/var/folders/jb/pfrlwvtn54x18ynnfy_9tkg00000gn/T/ipykernel_26727/2235184035.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labour_force_fact.loc[:, key_name] = labour_with_key[key_name].values
/var/folders/jb/pfrlwvtn54x18ynnfy_9tkg00000gn/T/ipykernel_26727/2235184035.py:7: SettingWithCopyWarning: 
A value i

In [ ]:
unemployed['WHYLEFT'] = unemployed['WHYLEFTO'].fillna(unemployed['WHYLEFTN'])
unemployed.drop(['WHYLEFTO', 'WHYLEFTN'], axis=1, inplace=True)

In [ ]:
# Handle NaN in Unemployment Dimension
# Fill the first NaN row with 'Not Applicable'
unemployed.iloc[0] = 'Not Applicable'
# Drop the rest of NaN rows
unemployed.dropna(how='all', inplace=True)
# Fill remaining NaN
unemployed.fillna("Not Applicable", inplace=True)

In [ ]:
# Elimate excessive mutual exclusive nulls within AGE_6 and AGE_12 columns by combining into one column
demographic['AGE'] = demographic['AGE_12'].fillna(demographic['AGE_6'])
demographic.drop(['AGE_12', 'AGE_6'], axis=1, inplace=True)


In [ ]:
unemployed = unemployed.rename(columns={'UNEMPLOYED_KEY': 'Unemployment Key', 'PRIORACT': 'Main activity before looking for work', 'YNOLOOK':'Reasons for not looking for work', 'LKATADS':'Unemployed looked at job ads', 'LKPUBAG':'Unemployed used public employment agency', 'FLOWUNEM':'Flows into unemployment', 'WHYLEFT':'Reason for job leave','UNEMFTPT': 'Job seeking status'})
unemployed

,Main activity before looking for work,Flows into unemployment,Unemployed used public employment agency,Unemployed looked at job ads,Reasons for not looking for work,Job seeking status,Unemployment Key,Reason for job leave
0,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
10,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,2,"Job losers, laid off"
12,Working,"Job losers, permanent layoff",Not Applicable,Not Applicable,Not Applicable,Full-time,3,"Job losers, laid off"
48,Working,Job leavers,Not Applicable,Yes,Not Applicable,Part-time,4,"Job leavers, own illness or disability"
61,Not Applicable,Not Applicable,Not Applicable,Not Applicable,"Wanted work, reason - awaiting recall or reply",Not Applicable,5,"Job losers, laid off"
...,...,...,...,...,...,...,...,...
305515,Other,"Re-entrants, worked 1 year ago or less",Yes,Yes,Not Applicable,Part-time,462,"Job losers, laid off"
307575,Working,Job leavers,Not Applicable,Yes,Not Applicable,Part-time,463,"Job leavers, personal or family responsibilities"
308353,Not Applicable,Not Applicable,Not Applicable,Not Applicable,"Wanted work, reason - own illness or disability",Not Applicable,464,"Job leavers, other reasons"
308757,Working,Future starts,Not Applicable,Yes,Not Applicable,Future starts,465,"Job leavers, other reasons"


In [ ]:
# Handle NaN in Job Dimension
job.fillna("Not Applicable", inplace=True)

In [ ]:
job = job.rename(columns={'JOB_KEY':'Job Key', 'NAICS_21':'Main Job Industry', 'NOC_10':'Employment Class', 'NOC_43':'Main Job Occupation', 'MJH': 'Single or Multiple Jobs', 'PERMTEMP':'Job Permanency Status', 'FIRMSIZE':'Firm Size', 'ESTSIZE':'Establishment Size','UNION':'Union Status','WHYPT':'Part Time Work Reason'})

job

,Main Job Industry,Employment Class,Main Job Occupation,Single or Multiple Jobs,Job Permanency Status,Establishment Size,Firm Size,Union Status,Part Time Work Reason,Job Key
0,Agriculture,9.0,"Workers and labourers in natural resources, ag...","Single jobholder, including job changers","Temporary, seasonal job",Less than 20 employees,Less than 20 employees,Non-unionized,Not Applicable,1
1,Public administration,5.0,Occupations in front-line public protection se...,"Single jobholder, including job changers",Permanent,20 to 99 employees,100 to 500 employees,Union member,Not Applicable,2
2,Other services (except public administration),8.0,Technical trades and transportation officers a...,"Single jobholder, including job changers",Permanent,Less than 20 employees,Less than 20 employees,Non-unionized,Not Applicable,3
3,Health care and social assistance,7.0,Sales and service support occupations,Not Applicable,Permanent,20 to 99 employees,More than 500 employees,Not Applicable,Not Applicable,4
4,Construction,8.0,Technical trades and transportation officers a...,"Single jobholder, including job changers",Permanent,20 to 99 employees,20 to 99 employees,Union member,Not Applicable,5
...,...,...,...,...,...,...,...,...,...,...
313748,Public administration,2.0,Administrative occupations and transportation ...,"Single jobholder, including job changers",Permanent,20 to 99 employees,More than 500 employees,Union member,Other reasons,19738
313821,Construction,2.0,Administrative and financial support and suppl...,"Single jobholder, including job changers",Permanent,20 to 99 employees,More than 500 employees,Not a member but covered by a union contract o...,Not Applicable,19739
313833,Public administration,7.0,Occupations in sales and services,"Single jobholder, including job changers",Permanent,Less than 20 employees,100 to 500 employees,Non-unionized,Business conditions or could not find full-tim...,19740
313879,Wholesale trade,7.0,Sales and service representatives and other cu...,"Single jobholder, including job changers","Temporary, term or contract job",20 to 99 employees,20 to 99 employees,Non-unionized,Going to school,19741


In [ ]:
demographic = demographic.rename(columns={'DEMOGRAPHIC_KEY':'Demographic Key', 'AGE':'Age Group', 'SEX': 'Sex', 'MARSTAT':'Martial Status', 'EDUC':'Highest Education Attained', 'SCHOOLN':'Student Status', 'IMMIG':'Immigration Status', 'EFAMTYPE':'Economic Family', 'AGYOWNK':'Youngest Child Age', 'TAXBRAC':'Tax Bracket'})
demographic

,Sex,Martial Status,Highest Education Attained,Student Status,Immigration Status,Economic Family,Youngest Child Age,Tax Bracket,Demographic Key,Age Group
0,Female,"Single, never married",0 to 8 years,Non-student,Non-immigrant,Person not in an economic family,Youngest child less than 6 years,2.0,1,45 to 49 years
1,Female,"Single, never married",Bachelor's degree,Non-student,Non-immigrant,Person not in an economic family,Youngest child less than 6 years,3.0,2,30 to 34 years
2,Male,Married,Some high school,Non-student,Non-immigrant,"Single-earner couple, male employed, no childr...",Youngest child less than 6 years,2.0,3,60 to 64 years
3,Female,Married,Postsecondary certificate or diploma,Non-student,Non-immigrant,"Non-earner couple, no children or none under 25",Youngest child less than 6 years,2.0,4,70 and over
4,Male,Married,High school graduate,Non-student,Non-immigrant,Other families,Youngest child less than 6 years,2.0,5,35 to 39 years
...,...,...,...,...,...,...,...,...,...,...
313829,Male,Living in common-law,Some postsecondary,Non-student,Non-immigrant,"Single-earner couple, male employed, youngest ...",Youngest child 13 to 17 years,2.0,29015,35 to 39 years
313830,Female,Married,0 to 8 years,Non-student,Non-immigrant,"Dual-earner couple, youngest child 0 to 17",Youngest child 6 to 12 years,1.0,29016,40 to 44 years
313839,Male,Living in common-law,High school graduate,Non-student,Non-immigrant,"Non-earner couple, youngest child 0 to 17",Youngest child less than 6 years,2.0,29017,35 to 39 years
313908,Male,Married,Postsecondary certificate or diploma,Non-student,"Immigrant, landed more than 10 years earlier","Non-earner couple, youngest child 0 to 17",Youngest child less than 6 years,2.0,29018,25 to 29 years


In [ ]:
date = date.rename(columns={'DATE_KEY':'Date Key', 'SURVMNTH': 'Month', 'SURVYEAR':'Year'})
date

,Year,Month,Date Key
0,2023,January,1
108064,2023,February,2
212915,2023,March,3


In [ ]:
geographic = geographic.rename(columns={'GEOGRAPHIC_KEY':'Geographic Key', 'PROV': 'Province', 'CMA':'Census Metropolitan Area'})
geographic

,Province,Census Metropolitan Area,Geographic Key
0,Quebec,MontrÈal,1
1,Saskatchewan,Other CMA or non-CMA,2
2,Newfoundland and Labrador,Other CMA or non-CMA,3
3,Ontario,Other CMA or non-CMA,4
6,British Columbia,Vancouver,5
7,Ontario,Toronto,6
17,New Brunswick,Other CMA or non-CMA,7
18,Quebec,Other CMA or non-CMA,8
23,Manitoba,Winnipeg,9
32,Manitoba,Other CMA or non-CMA,10


In [ ]:
labour_force_fact = labour_force_fact.rename(columns={'REC_NUM':'Labour Force Key', 'DATE_KEY': 'Date Key', 'GEOGRAPHIC_KEY':'Geographic Key', 'DEMOGRAPHIC_KEY':'Demographic Key','JOB_KEY':'Job Key', 'UNEMPLOYED_KEY': 'Unemployment Key', 'LFSSTAT':'Labour Force Status', 'HRLYEARN': 'Usual Hourly Wages', 'UHRSMAIN': 'Usual Hours worked per week at main job', 'AHRSMAIN':'Actual Hours worked per week at main job', 'ATOTHRS':'Actual Hours worked per week at all job', 'UTOTHRS':'Usual Hours worked per week at all jobs', 'HRSAWAY':'Hours away from work','PAIDOT':'Paid overtime hours in reference week', 'XTRAHRS':'Number of overtime or extra hours worked', 'WKSAWAY':'Numbers of weeks absent from work', 'YABSENT':'Reason of absence','PREVTEN':'Previous job tenure','TENURE':'Current job tenure','DURUNEMP':'Duration of unemployment', 'WEEKINCOME':'Weekly income', 'ANNUALINCOME': 'Annual income'})
labour_force_fact

,Labour Force Key,Labour Force Status,Usual Hourly Wages,Usual Hours worked per week at main job,Actual Hours worked per week at main job,Usual Hours worked per week at all jobs,Actual Hours worked per week at all job,Hours away from work,Numbers of weeks absent from work,Paid overtime hours in reference week,...,Duration of unemployment,Current job tenure,Previous job tenure,Weekly income,Annual income,Date Key,Geographic Key,Demographic Key,Job Key,Unemployment Key
0,1,1,20.000000,400.000000,400.000000,40.000000,400.000000,0.000000,13.791588,0.000000,...,15.261926,107.000000,58.344636,800.00,41600.000000,1,1,1,1,1
1,2,1,42.380000,425.000000,425.000000,42.500000,425.000000,0.000000,13.791588,0.000000,...,15.261926,39.000000,58.344636,1801.15,93659.800000,1,2,2,2,1
2,3,1,19.000000,400.000000,400.000000,40.000000,400.000000,0.000000,13.791588,0.000000,...,15.261926,129.000000,58.344636,760.00,39520.000000,1,3,3,3,1
3,4,4,32.462154,355.900551,321.877326,36.358793,328.741559,14.705105,13.791588,8.133639,...,15.261926,96.045098,58.344636,1000.00,62935.340129,1,4,4,4,1
4,5,1,34.830000,400.000000,400.000000,40.000000,400.000000,0.000000,13.791588,0.000000,...,15.261926,188.000000,58.344636,1393.20,72446.400000,1,4,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313953,101039,3,32.462154,355.900551,321.877326,36.358793,328.741559,14.705105,13.791588,8.133639,...,10.000000,96.045098,58.344636,1000.00,62935.340129,3,15,1157,4,216
313954,101040,1,23.280000,400.000000,400.000000,40.000000,400.000000,0.000000,13.791588,0.000000,...,15.261926,240.000000,58.344636,931.20,48422.400000,3,9,1842,1277,1
313955,101041,1,64.100000,375.000000,375.000000,37.500000,375.000000,0.000000,13.791588,0.000000,...,15.261926,240.000000,58.344636,2403.75,124995.000000,3,8,9629,8613,1
313956,101042,1,32.970000,350.000000,350.000000,35.000000,350.000000,0.000000,13.791588,0.000000,...,15.261926,8.000000,58.344636,1153.95,60005.400000,3,6,1199,13345,1


In [ ]:
config = {
      'host':'meow.mysql.database.azure.com',
      'user':'nickisthebest',
      'password':'Asdf123$',
      'database':'canadian_labour'
}
'''
try:
    conn = mysql.connector.connect(**config)
    print("Connection established")
    cursor = conn.cursor()

    #cursor.execute("DROP TABLE IF EXISTS LABOUR_FORCE_FACT;")

    for index, row in date.iterrows():
        row = row.where(pd.notnull(row), None)
        cursor.execute("INSERT INTO DATE_DIMENSION (SURVYEAR, SURVMNTH, DATE_KEY) VALUES (%s, %s, %s)",
                        (row.SURVYEAR, row.SURVMNTH, row.DATE_KEY))

    print("ddone")
    conn.commit()

    for index, row in geo.iterrows():
        row = row.where(pd.notnull(row), None)
        cursor.execute("INSERT INTO GEOGRAPHIC_DIMENSION (PROV, CMA,GEOGRAPHIC_KEY) VALUES (%s, %s, %s)",
                        (row.PROV, row.CMA, row.GEOGRAPHIC_KEY))
    print("gdone")
    conn.commit()

    for index, row in job.iterrows():
        row = row.where(pd.notnull(row), None)
        cursor.execute("INSERT INTO JOB_DIMENSION (NAICS_21, NOC_10, NOC_43, ESTSIZE, FIRMSIZE, WHYPT, JOB_KEY) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                        (row.NAICS_21, row.NOC_10, row.NOC_43, row.ESTSIZE, row.FIRMSIZE, row.WHYPT, row.JOB_KEY))
    print("jdone")
    conn.commit()

    for index, row in u.iterrows():
        row = row.where(pd.notnull(row), None)
        cursor.execute("INSERT INTO UNEMPLOYED_DIMENSION (PRIORACT, FLOWUNEM, LKPUBAG, LKEMPLOY, LKRELS, LKATADS, WHYLEFTO, WHYLEFTN, YNOLOOK, UNEMFTPT, UNEMPLOYED_KEY) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        (row.PRIORACT, row.FLOWUNEM, row.LKPUBAG, row.LKEMPLOY, row.LKRELS, row.LKATADS, row.WHYLEFTO, row.WHYLEFTN, row.YNOLOOK, row.UNEMFTPT, row.UNEMPLOYED_KEY))
    print("udone")
    conn.commit()

    for index, row in demo.iterrows():
        row = row.where(pd.notnull(row), None)
        cursor.execute("INSERT INTO DEMOGRAPHIC_DIMENSION (SEX, MARSTAT, EDUC, IMMIG, EFAMTYPE, AGYOWNK, AGE, TAXBRAC, DEMOGRAPHIC_KEY) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        (row.SEX, row.MARSTAT, row.EDUC, row.IMMIG, row.EFAMTYPE, row.AGYOWNK, row.AGE, row.TAXBRAC, row.DEMOGRAPHIC_KEY))
    print("demo")
    conn.commit()

    for index, row in l.iterrows():
        row = row.where(pd.notnull(row), None)
        cursor.execute("INSERT INTO LABOUR_FORCE_FACT (REC_NUM, LFSSTAT, HRLYEARN, UHRSMAIN, AHRSMAIN, UTOTHRS, ATOTHRS, HRSAWAY, WKSAWAY, PAIDOT, XTRAHRS, YABSENT, DURUNEMP, TENURE, PREVTEN, DATE_KEY, GEOGRAPHIC_KEY, DEMOGRAPHIC_KEY, JOB_KEY, UNEMPLOYED_KEY) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        (row.REC_NUM, row.LFSSTAT, row.HRLYEARN, row.UHRSMAIN, row.AHRSMAIN, row.UTOTHRS, row.ATOTHRS, row.HRSAWAY, row.WKSAWAY, row.PAIDOT, row.XTRAHRS, row.YABSENT, row.DURUNEMP, row.TENURE, row.PREVTEN, row.DATE_KEY, row.GEOGRAPHIC_KEY, row.DEMOGRAPHIC_KEY, row.JOB_KEY, row.UNEMPLOYED_KEY))
        if index % 10000 == 0:
            print(index)
    conn.commit()
    cursor.close()
    conn.close()
    print("Data inserted successfully")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with the user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor = conn.cursor()

conn.commit()
cursor.close()
conn.close()
print("SUCESSFUL LOAD PROCESS")
'''

'\ntry:\n    conn = mysql.connector.connect(**config)\n    print("Connection established")\n    cursor = conn.cursor()\n\n    #cursor.execute("DROP TABLE IF EXISTS LABOUR_FORCE_FACT;")\n\n    for index, row in date.iterrows():\n        row = row.where(pd.notnull(row), None)\n        cursor.execute("INSERT INTO DATE_DIMENSION (SURVYEAR, SURVMNTH, DATE_KEY) VALUES (%s, %s, %s)",\n                        (row.SURVYEAR, row.SURVMNTH, row.DATE_KEY))\n\n    print("ddone")\n    conn.commit()\n\n    for index, row in geo.iterrows():\n        row = row.where(pd.notnull(row), None)\n        cursor.execute("INSERT INTO GEOGRAPHIC_DIMENSION (PROV, CMA,GEOGRAPHIC_KEY) VALUES (%s, %s, %s)",\n                        (row.PROV, row.CMA, row.GEOGRAPHIC_KEY))\n    print("gdone")\n    conn.commit()\n\n    for index, row in job.iterrows():\n        row = row.where(pd.notnull(row), None)\n        cursor.execute("INSERT INTO JOB_DIMENSION (NAICS_21, NOC_10, NOC_43, ESTSIZE, FIRMSIZE, WHYPT, JOB_KEY) VALUES 